## Import

In [1]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotInteractableException
from selenium import webdriver
from tqdm import tqdm
from bs4 import BeautifulSoup

import requests
import streamlit as st
import pandas as pd
import numpy as np
import OpenDartReader
import warnings
import time, datetime
import re, os

warnings.filterwarnings('ignore')

* functions

In [2]:
def set_num(x):
    if type(x) != str:
        return x
    else:
        return float(re.sub("[^0-9]", "", x))
    
def get_kind(driver, code, FIRST = True):
    ## 01. 회사명 검색하기
    name_element = driver.find_element(By.ID, 'AKCKwd')
    name_element.click()
    name_element.clear()
    time.sleep(0.1)
    name_element.send_keys(code)

    if FIRST:
        ## 02. 신규상장만 활용
        check_box = '/html/body/section[2]/section/form/section/div/div[1]/table/tbody/tr[7]/td/label[{}]'

        for idx in range(4):
            driver.find_element(By.XPATH, check_box.format(idx+2)).click()

        time.sleep(0.1)

    ## 03. 기간 전체 설정
    driver.find_element(By.CLASS_NAME, 'ord-07').click()

    ## 04. 검색 시작
    search_element = driver.find_element(By.CLASS_NAME, 'btn-sprite.type-00.vmiddle.search-btn')
    search_element.click()
    time.sleep(1.5)

    ### 모든 데이터 수집
    temp_df = pd.read_html(driver.page_source)
    listing_df = [x for x in temp_df if "회사명" in x and "상장유형" in x][0]

    if listing_df.shape[0] != 0:
        driver.find_element(By.CSS_SELECTOR, '#main-contents > section.scrarea.type-00 > table > tbody > tr').click()
        time.sleep(0.25)

        driver.switch_to.window(driver.window_handles[1])
        wait = WebDriverWait(driver, 10, poll_frequency=0.25)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "btn-sprite.type-98.vmiddle")))
        
        time.sleep(0.5)
        # 상장주식수
        l_cnt = driver.find_element(By.CSS_SELECTOR, '#tab-contents > table:nth-child(3) > tbody > tr:nth-child(9) > td:nth-child(2)')
        l_cnt = int(re.sub("[^0-9]", "", l_cnt.text))

        # 유통가능주식수
        c_cnt = driver.find_element(By.CSS_SELECTOR, '#tab-contents > table:nth-child(3) > tbody > tr:nth-child(10) > td.txr')
        c_cnt = int(re.sub("[^0-9]", "", c_cnt.text))

        # 주요 제품
        m_product = driver.find_element(By.CSS_SELECTOR, "#tab-contents > table:nth-child(3) > tbody > tr:nth-child(6) > td").text

        try:
            # switch tab
            driver.find_element(By.XPATH, '/html/body/form/section/nav/ul/li[2]/a').click()
            time.sleep(0.25)

            wait = WebDriverWait(driver, 10, poll_frequency=0.25)
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#tab-contents > table:nth-child(15) > tbody > tr:nth-child(2) > td:nth-child(2)")))
            
        except ElementNotInteractableException:
            col_names = [x+"매출액" + "({})".format(y) for x, y in zip(['전전연도', '직전연도', '당해연도'], ['T-2', 'T-1', 'T'])]
            col_names.extend([x+"영업이익" + "({})".format(y) for x, y in zip(['전전연도', '직전연도', '당해연도'], ['T-2', 'T-1', 'T'])])
            col_names.extend([x+"당기순이익" + "({})".format(y) for x, y in zip(['전전연도', '직전연도', '당해연도'], ['T-2', 'T-1', 'T'])])

            listing_df['종목코드'] = code
            listing_df['상장주식수'] = l_cnt
            listing_df['유통가능주식수'] = c_cnt
            listing_df['주요제품'] = m_product
            listing_df.index = [0]

            df_change = pd.DataFrame.from_dict({x:['-'] for x in col_names})
            df_change.index = [0]

            second_df = pd.concat([listing_df, df_change], axis = 1)
            
            driver.close()
            driver.switch_to.window(driver.window_handles[0])

            
            return second_df

        dfs = pd.read_html(driver.page_source, header=0)

        try:
            get_idx = [idx for idx, x in enumerate(dfs) if "매출액(수익)" in list(x['항목']) and "영업이익(손실)" in list(x['항목']) and "당기순이익(손실)" in list(x['항목'])][0]
        except:
            get_idx = 1
            
        df = dfs[get_idx]
        df = df.loc[df['항목'].isin(['매출액(수익)', '영업이익(손실)', '당기순이익(손실)'])]

        if df.shape[0] != 3:
            except_values = [x for x in ['매출액(수익)', '영업이익(손실)', '당기순이익(손실)'] if x not in list(df['항목'])]
            append_df = pd.DataFrame({"항목":[x for x in except_values],
                          df.columns[1]:['-' for x in range(len(except_values))],
                          df.columns[2]:['-' for x in range(len(except_values))],
                          df.columns[3]:['-' for x in range(len(except_values))]})
            df = pd.concat([df, append_df])
            df['항목'] = df['항목'].astype("category")
            df['항목'] = df['항목'].cat.set_categories(['매출액(수익)', '영업이익(손실)', '당기순이익(손실)'])
            df = df.sort_values("항목")

        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        
        col_names = [x+"매출액" + "({})".format(y) for x, y in zip(['전전연도', '직전연도', '당해연도'], ['T-2', 'T-1', 'T'])]
        col_names.extend([x+"영업이익" + "({})".format(y) for x, y in zip(['전전연도', '직전연도', '당해연도'], ['T-2', 'T-1', 'T'])])
        col_names.extend([x+"당기순이익" + "({})".format(y) for x, y in zip(['전전연도', '직전연도', '당해연도'], ['T-2', 'T-1', 'T'])])
        
        listing_df['종목코드'] = code
        listing_df['상장주식수'] = l_cnt
        listing_df['유통가능주식수'] = c_cnt
        listing_df['주요제품'] = m_product
        listing_df.index = [0]
        
        df_change = pd.DataFrame.from_dict({x:[y] for x,y in zip(col_names, np.array(df.iloc[:, 1:]).reshape(1, -1)[0])})
        df_change.index = [0]
        
        second_df = pd.concat([listing_df, df_change], axis = 1)
        
    return second_df

In [3]:
def change_corp(x):
    if "모간스탠리" in x : return "MS"
    elif "골드" in x : return "골드만"
    elif "씨티" in x : return "씨티"
    elif "메릴" in x : return "메릴린치"
    elif "케이비" in x : return "KB"
    elif "아이비케이" in x : return "IBK"
    elif "에스케이" in x : return "SK"
    elif "디비금융" in x : return "DB"
    else: return x.replace("투자", "").replace("금융", "").replace("증권", "").replace("에셋", "").replace("(주)", "").replace("㈜", "")

def get_case(x):
    if "공동대표" in x: return "공동대표"
    elif "공동주관" in x: return "공동"
    elif "대표" in x: return "대표"
    elif "인수" in x : return "인수"
    else: return "대표"

* API Setting

In [4]:
api_key = '1b39652cef07f626c9d37375edf582ee51b1407f'
dart = OpenDartReader(api_key)

* 최종 결과물 형태 제작

In [5]:
results_df = pd.DataFrame(np.empty((0, 25)))

col_names = [x+"매출액" + "({})".format(y) for x, y in zip(['전전연도', '직전연도', '당해연도'], ['T-2', 'T-1', 'T'])]
col_names.extend([x+"영업이익" + "({})".format(y) for x, y in zip(['전전연도', '직전연도', '당해연도'], ['T-2', 'T-1', 'T'])])
col_names.extend([x+"당기순이익" + "({})".format(y) for x, y in zip(['전전연도', '직전연도', '당해연도'], ['T-2', 'T-1', 'T'])])

tot_cols = ['수요예측(시작일)', '수요예측(종료일)', '상장예정일', '대표주관사', '회사명', '공모주식수',
            '신주모집수', '구주매출', '하단공모가액', '상단공모가액', '상장예정주식수', '유통주식수', '확정공모가액',
            '수요예측경쟁률', '확약률']
tot_cols.extend(col_names)
tot_cols.extend(['주요제품'])

results_df.columns = tot_cols

* 38.co.kr - 외부 수집

In [6]:
start_dt = '2023-07-01'
end_dt = '2023-07-31'

In [7]:
cnt = 0
max_page = 20

for page in range(1, max_page+1):
    outer_url = 'http://www.38.co.kr/html/fund/index.htm?o=r1&page={}'.format(page)
    base_url = 'http://www.38.co.kr/html/fund'
    
    response = requests.get(outer_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    tb_src = soup.select('table[summary="수요예측결과"]')
    
    temp_df = pd.read_html(str(tb_src))[0]
    temp_df = temp_df.loc[~temp_df['기업명'].isna()]
    
    if cnt == 0:
        url_tags = soup.select('tbody > tr > td > a')
        temp_df['url'] = [base_url + x.attrs['href'].split(".")[-1] for x in url_tags]
        outer_df = temp_df

    else:
        url_tags = soup.select('tbody > tr > td > a')
        temp_df['url'] = [base_url + x.attrs['href'].split(".")[-1] for x in url_tags]
        outer_df = pd.concat([outer_df, temp_df])
        
    cnt += 1
    
outer_df = outer_df.loc[(outer_df['예측일'] >= start_dt.replace("-", ".")) & (outer_df['예측일'] <= end_dt.replace("-", "."))]

In [8]:
outer_df.head()

,기업명,예측일,공모희망가(원),공모가(원),공모금액 (백만원),기관 경쟁률,의무보유 확약,주간사,url
1,뷰티스킨,2023.07.10,"21,000~24,000",26000.0,-,1819.7 :1,10.93%,DB금융투자,http://www.38.co.kr/html/fund/?o=v&no=1942&l=&...
2,SK증권스팩9호,2023.07.06,"2,000~2,000",2000.0,8000,"1,002.51:1",3.54%,SK증권,http://www.38.co.kr/html/fund/?o=v&no=1953&l=&...
3,센서뷰,2023.07.03,"2,900~3,600",4500.0,11310,"1,673.33:1",7.41%,삼성증권,http://www.38.co.kr/html/fund/?o=v&no=1936&l=&...
4,와이랩,2023.07.03,"7,000~8,000",9000.0,21000,1821.64:1,14.72%,한국투자증권,http://www.38.co.kr/html/fund/?o=v&no=1939&l=&...


* 38.co.kr - 내부 수집

In [9]:
def get_38df(soup, x):
    return pd.read_html(str(soup.select('table[summary="{}"]'.format(x))[0]))[0]

def get_values(soup):
    table_src = soup.select('table[bgcolor="#C2D2DF"]')
    table_src = [str(x) for x in table_src]
    table_src2 = [x for x in table_src if ("자산가치" in x) or ("수익가치" in x) or ("본질가치" in x)]
    dfs = [pd.read_html(x)[0] for x in table_src2]
    
    if len(dfs) > 0:
        return dfs
    else:
        return [None, None, None]

In [14]:
data_dict = {x:[] for x in ['종목코드', '수요예측(시작일)', '수요예측(종료일)', '상장예정일', '기업명', '공모주식수', '신주모집수','구주매출', '하단공모가액', '상단공모가액', '확정공모가액']}
    
for name, url in zip(outer_df['기업명'], outer_df['url']):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    overview_df = get_38df(soup, '기업개요')
    overview_df.columns = ["O" + str(x) for x in range(overview_df.shape[1])]
    
    inform_df = get_38df(soup, '공모정보')
    inform_df.columns = ["I" + str(x) for x in range(inform_df.shape[1])]
    
    cal_df = get_38df(soup, '공모청약일정')
    cal_df.columns = ["C" + str(x) for x in range(cal_df.shape[1])]
    
    # 종목 코드
    code = overview_df.loc[overview_df.O2 == '종목코드', 'O3'].values[0]
    
    # 수요 예측일
    p_start, p_end = cal_df.loc[cal_df.C1 == '수요예측일', 'C2'].values[0].split(" ~ ")
    
    # 상장일
    l_date = cal_df.loc[cal_df.C1 == '상장일', 'C2'].values[0]
    
    # 공모주식수
    t_cnt = int(re.sub("[^0-9]", "",inform_df.loc[inform_df.I0 == '총공모주식수', 'I1'].values[0]))
    
    # 신주모집수, 구주모집수
    try:
        new_cnt, old_cnt = re.findall('(?<=:)(.*?)(?=주)', inform_df.loc[inform_df.I0 == '상장공모', 'I1'].values[0])
        if type(new_cnt) == str:
            new_cnt = int(re.sub("[^0-9]", "", new_cnt))
        if type(old_cnt) == str:
            old_cnt = int(re.sub("[^0-9]", "", old_cnt))
        
    except:
        new_cnt = re.findall('(?<=:)(.*?)(?=주)', inform_df.loc[inform_df.I0 == '상장공모', 'I1'].values[0])[0]
        if type(new_cnt) == str:
            new_cnt = int(re.sub("[^0-9]", "", new_cnt))
        old_cnt = 0
    
    # 하단공모가액, 상단공모가액
    lower, upper = [int(re.sub("[^0-9]", "", x)) for x in inform_df.loc[inform_df.I0 == '희망공모가액', 'I1'].values[0].split(" ~ ")]
    
    # 확정공모가액
    f_price = int(re.sub("[^0-9]", "", inform_df.loc[inform_df.I0 == '확정공모가', 'I1'].values[0]))
    
    data_dict['종목코드'].append(code)
    data_dict['수요예측(시작일)'].append(p_start)
    data_dict['수요예측(종료일)'].append(p_end)
    data_dict['상장예정일'].append(l_date)
    data_dict['기업명'].append(name)
    data_dict['공모주식수'].append(t_cnt)
    data_dict['신주모집수'].append(new_cnt)
    data_dict['구주매출'].append(old_cnt)
    data_dict['하단공모가액'].append(lower)
    data_dict['상단공모가액'].append(upper)
    data_dict['확정공모가액'].append(f_price)

right_df = outer_df.loc[:, ['기업명', '기관 경쟁률', '의무보유 확약', '공모금액 (백만원)']]
left_df = pd.DataFrame.from_dict(data_dict)
first_df = pd.merge(left_df, right_df, on = '기업명', how = 'inner')

In [11]:
today = datetime.datetime.today()
today = "{}.{:02d}.{}".format(today.year, today.month, today.day)

In [16]:
#first_df = first_df.loc[first_df['상장예정일'] <= today]
first_df = first_df.loc[first_df['수요예측(시작일)'] <= today]
first_df.index = range(0, first_df.shape[0])

In [17]:
first_df

,종목코드,수요예측(시작일),수요예측(종료일),상장예정일,기업명,공모주식수,신주모집수,구주매출,하단공모가액,상단공모가액,확정공모가액,기관 경쟁률,의무보유 확약,공모금액 (백만원)
0,406820,2023.07.10,2023.07.11,2023.07.24,뷰티스킨,440000,440000,0,21000,24000,26000,1819.7 :1,10.93%,-
1,455910,2023.07.06,2023.07.07,2023.07.21,SK증권스팩9호,4000000,4000000,0,2000,2000,2000,"1,002.51:1",3.54%,8000
2,321370,2023.07.03,2023.07.04,2023.07.19,센서뷰,3900000,3900000,0,2900,3600,4500,"1,673.33:1",7.41%,11310
3,432430,2023.07.03,2023.07.04,2023.07.20,와이랩,3000000,3000000,0,7000,8000,9000,1821.64:1,14.72%,21000


* KIND

In [ ]:
1. 예비심사기업에서 종목 코드로 조회하여 회사명을 딴다
2. 공모기업현황에서 해당 회사명을 클릭한다
3. 새 창에서 기존에 가져왔던 모든 내용을 수집한다.

In [16]:
'''
options = Options()
options.add_argument('--disable-gpu')
options.add_argument('--headless')

@st.experimental_singleton
def get_driver():
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
'''

"\noptions = Options()\noptions.add_argument('--disable-gpu')\noptions.add_argument('--headless')\n\n@st.experimental_singleton\ndef get_driver():\n\xa0\xa0\xa0 return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)\n"

In [18]:
driver = webdriver.Chrome()

In [19]:
## 01.KIND 접속
driver.get("https://kind.krx.co.kr/listinvstg/listingcompany.do?method=searchListingTypeMain")

wait = WebDriverWait(driver, 10, poll_frequency=0.25)
wait.until(EC.presence_of_element_located((By.ID, "fromDate")))

for idx, code in enumerate(first_df['종목코드']):
    if idx == 0:
        second_df = get_kind(driver, code)
    else:
        df = get_kind(driver, code, FIRST = False)
        second_df = pd.concat([second_df, df])
        
driver.close()
del second_df['회사명'], second_df['상장일'], second_df['상장유형'], second_df['업종'], second_df['국적'], second_df['증권구분']

second_df.head(1)

IndexError: list index out of range

In [20]:
final_df = pd.merge(first_df, second_df, on = '종목코드', how = 'left')

In [21]:
del final_df['종목코드']

* 최종 결과 형태

In [23]:
final_df.columns

Index(['수요예측(시작일)', '수요예측(종료일)', '상장예정일', '기업명', '공모주식수', '신주모집수', '구주매출',
       '하단공모가액', '상단공모가액', '확정공모가액', '기관 경쟁률', '의무보유 확약', '공모금액 (백만원)',
       '상장주선인/ 지정자문인', '상장주식수', '유통가능주식수', '주요제품', '전전연도매출액(T-2)',
       '직전연도매출액(T-1)', '당해연도매출액(T)', '전전연도영업이익(T-2)', '직전연도영업이익(T-1)',
       '당해연도영업이익(T)', '전전연도당기순이익(T-2)', '직전연도당기순이익(T-1)', '당해연도당기순이익(T)'],
      dtype='object')

In [24]:
select_col = ['수요예측(시작일)', '수요예측(종료일)', '상장예정일', '상장주선인/ 지정자문인', '기업명', 
              '공모주식수', '신주모집수', '구주매출', '하단공모가액', '상단공모가액', '상장주식수',
              '유통가능주식수', '확정공모가액', '기관 경쟁률', '의무보유 확약', 
               '전전연도매출액(T-2)', '직전연도매출액(T-1)', '당해연도매출액(T)', '전전연도영업이익(T-2)', '직전연도영업이익(T-1)',
               '당해연도영업이익(T)', '전전연도당기순이익(T-2)', '직전연도당기순이익(T-1)', '당해연도당기순이익(T)', '주요제품']

In [111]:
final_df.loc[:, select_col].to_excel("IB전략컨설팅부-IPO공모기업현황요약.xlsx", index = False)